# Chapter 8: Building Your First LLM Application - Solutions
**From: Zero to AI Agent**

Try the exercises in the main notebook first before viewing these solutions!

In [ ]:
# Setup
!pip install -q -r requirements.txt

from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("Setup complete!")

---
## Exercise 8.1.1: Personality Bot

Create a chatbot that can switch between different personalities (pirate, chef, poet, robot).

In [ ]:
# Define personalities
personalities = {
    "pirate": "You are a friendly pirate. Speak with pirate slang, say 'arr' and 'matey' often.",
    "chef": "You are a passionate French chef. Use cooking metaphors and French words.",
    "poet": "You are a romantic poet. Speak in a lyrical manner, sometimes in rhyme.",
    "robot": "You are a helpful robot. Be logical, occasionally say 'BEEP BOOP'."
}

class PersonalityBot:
    def __init__(self):
        self.current_personality = "pirate"
        self.conversation = [{"role": "system", "content": personalities[self.current_personality]}]
    
    def switch_personality(self, personality):
        if personality in personalities:
            self.current_personality = personality
            self.conversation = [{"role": "system", "content": personalities[personality]}]
            return f"Switched to {personality}!"
        return f"Unknown. Choose from: {list(personalities.keys())}"
    
    def chat(self, message):
        self.conversation.append({"role": "user", "content": message})
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.conversation,
            temperature=0.9
        )
        ai_message = response.choices[0].message.content
        self.conversation.append({"role": "assistant", "content": ai_message})
        if len(self.conversation) > 10:
            self.conversation = [self.conversation[0]] + self.conversation[-9:]
        return ai_message

# Demo
bot = PersonalityBot()
print(f"Current: {bot.current_personality}")
print(bot.chat("Tell me about your day"))
print("\n" + bot.switch_personality("robot"))
print(bot.chat("Tell me about your day"))

---
## Exercise 8.1.2: Conversation Saver

Create a chat that can save and load conversations to/from files.

In [ ]:
import json
from datetime import datetime

class ConversationSaver:
    def __init__(self):
        self.conversation = [{"role": "system", "content": "You are a helpful assistant."}]
        self.message_count = 0
    
    def save(self):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        # Save as JSON for loading later
        json_file = f"conversation_{timestamp}.json"
        with open(json_file, 'w') as f:
            json.dump({"messages": self.conversation, "count": self.message_count}, f, indent=2)
        return json_file
    
    def load(self, filename):
        with open(filename, 'r') as f:
            data = json.load(f)
            self.conversation = data["messages"]
            self.message_count = data["count"]
        return f"Loaded {self.message_count} messages"
    
    def chat(self, message):
        self.conversation.append({"role": "user", "content": message})
        self.message_count += 1
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.conversation
        )
        ai_message = response.choices[0].message.content
        self.conversation.append({"role": "assistant", "content": ai_message})
        self.message_count += 1
        return ai_message

# Demo
saver = ConversationSaver()
print(saver.chat("Hello, my name is Bob"))
saved_file = saver.save()
print(f"Saved to: {saved_file}")

---
## Exercise 8.1.3: Cost Calculator

Create a chat that tracks and displays API costs in real-time.

In [ ]:
class CostTrackerChat:
    def __init__(self, model="gpt-3.5-turbo"):
        self.model = model
        self.conversation = [{"role": "system", "content": "You are helpful."}]
        self.total_cost = 0.0
        self.call_count = 0
        self.pricing = {
            "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},
            "gpt-4": {"input": 0.03, "output": 0.06}
        }
    
    def chat(self, message):
        self.conversation.append({"role": "user", "content": message})
        response = client.chat.completions.create(
            model=self.model,
            messages=self.conversation
        )
        ai_message = response.choices[0].message.content
        self.conversation.append({"role": "assistant", "content": ai_message})
        
        # Calculate cost
        rates = self.pricing.get(self.model, self.pricing["gpt-3.5-turbo"])
        cost = (response.usage.prompt_tokens / 1000 * rates["input"]) + \
               (response.usage.completion_tokens / 1000 * rates["output"])
        self.total_cost += cost
        self.call_count += 1
        
        return {"response": ai_message, "cost": cost, "total": self.total_cost}

# Demo
tracker = CostTrackerChat()
result = tracker.chat("What is Python?")
print(f"Response: {result['response'][:80]}...")
print(f"This call: ${result['cost']:.6f} | Total: ${result['total']:.6f}")

---
## Exercise 8.2.1: Temperature Tester

Create a tool that generates responses at different temperature settings.

In [ ]:
def test_temperatures(prompt, temperatures=[0, 0.7, 1.5]):
    print(f"Prompt: '{prompt}'\n" + "=" * 50)
    
    for temp in temperatures:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=temp,
            max_tokens=100
        )
        label = "Focused" if temp == 0 else "Balanced" if temp == 0.7 else "Creative"
        print(f"\nTemp {temp} ({label}):")
        print(response.choices[0].message.content)

# Demo
test_temperatures("Write a one-sentence description of a sunset")

---
## Exercise 8.2.2: Conversation Counter

Build a chatbot that counts messages and tracks statistics.

In [ ]:
class ConversationCounter:
    def __init__(self):
        self.message_count = 0
        self.total_words = 0
        self.estimated_cost = 0.0
    
    def chat(self, message):
        self.message_count += 1
        self.total_words += len(message.split())
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": message}]
        )
        
        ai_response = response.choices[0].message.content
        self.total_words += len(ai_response.split())
        self.estimated_cost += (response.usage.total_tokens / 1000) * 0.002
        
        return {"response": ai_response, "words": len(ai_response.split()), "cost": self.estimated_cost}
    
    def stats(self):
        return {"messages": self.message_count, "words": self.total_words, "cost": self.estimated_cost}

# Demo
counter = ConversationCounter()
result = counter.chat("Hello!")
print(f"Response: {result['response']}")
print(f"Stats: {counter.stats()}")

---
## Exercise 8.2.3: Personality Switcher

Create a chatbot with multiple personalities with different temperature settings.

In [ ]:
personalities = {
    "teacher": {"prompt": "You are a patient teacher.", "temp": 0.3},
    "friend": {"prompt": "You are a casual friend.", "temp": 0.9},
    "coach": {"prompt": "You are an energetic coach.", "temp": 1.2}
}

class PersonalitySwitcher:
    def __init__(self):
        self.current = "teacher"
        self.conversation = [{"role": "system", "content": personalities[self.current]["prompt"]}]
    
    def switch(self, personality):
        if personality in personalities:
            self.current = personality
            self.conversation = [{"role": "system", "content": personalities[personality]["prompt"]}]
            return f"Switched to {personality}"
        return "Unknown personality"
    
    def chat(self, message):
        self.conversation.append({"role": "user", "content": message})
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.conversation,
            temperature=personalities[self.current]["temp"]
        )
        ai_response = response.choices[0].message.content
        self.conversation.append({"role": "assistant", "content": ai_response})
        return ai_response

# Demo
switcher = PersonalitySwitcher()
print(switcher.chat("How do I learn Python?"))

---
## Exercise 8.3.1: Token Predictor

Build a tool that predicts token counts and compares to actual counts.

In [ ]:
def predict_tokens(text):
    char_est = len(text) // 4
    word_est = int(len(text.split()) * 1.3)
    return {'char': char_est, 'word': word_est, 'avg': (char_est + word_est) // 2}

def test_prediction(prompt):
    est = predict_tokens(prompt)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    actual = response.usage.prompt_tokens
    error = abs(est['avg'] - actual)
    
    print(f"Predicted: {est['avg']}, Actual: {actual}, Error: {error}")
    return error

# Demo
test_prediction("What is the capital of France?")

---
## Exercise 8.3.2: Response Timer

Create a tool that tests how prompt length affects response time.

In [ ]:
import time

def time_api_call(prompt):
    start = time.time()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    elapsed = time.time() - start
    return elapsed, response.usage.total_tokens

# Test different lengths
prompts = [("Short", "Hi"), ("Medium", "Write a haiku"), ("Long", "Explain relativity in detail")]

for name, prompt in prompts:
    elapsed, tokens = time_api_call(prompt)
    print(f"{name}: {elapsed:.2f}s, {tokens} tokens, {tokens/elapsed:.0f} tok/s")

---
## Exercise 8.3.3: Model Comparison

Build a tool that compares responses from different models.

In [ ]:
def compare_models(prompt, models=["gpt-3.5-turbo"]):
    print(f"Prompt: '{prompt}'\n")
    for model in models:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100
            )
            tokens = response.usage.total_tokens
            cost = tokens * 0.002 / 1000 if 'gpt-3.5' in model else tokens * 0.03 / 1000
            print(f"{model}: {tokens} tokens, ${cost:.6f}")
            print(f"  {response.choices[0].message.content[:80]}...\n")
        except Exception as e:
            print(f"{model}: Error - {e}")

# Demo
compare_models("Write a haiku")

---
## Exercise 8.4.1: Error Logger

Create an error logging system for API errors.

In [ ]:
from datetime import datetime
from collections import Counter

class ErrorLogger:
    def __init__(self):
        self.errors = []
    
    def log(self, error, context=""):
        self.errors.append({
            'time': datetime.now().isoformat(),
            'type': type(error).__name__,
            'message': str(error)[:100],
            'context': context
        })
    
    def report(self):
        if not self.errors:
            return "No errors logged"
        counts = Counter(e['type'] for e in self.errors)
        return f"Errors: {len(self.errors)}\n" + "\n".join(f"  {t}: {c}" for t, c in counts.items())

# Demo
logger = ErrorLogger()
try:
    raise ValueError("Test error")
except Exception as e:
    logger.log(e, "Testing")
print(logger.report())

---
## Exercise 8.4.2: Resilient Caller

Build an adaptive API caller with health-based parameter adjustment.

In [ ]:
class ResilientCaller:
    def __init__(self):
        self.health = 100
        self.errors = 0
        self.successes = 0
    
    def call(self, messages):
        # Adjust params based on health
        max_tokens = None if self.health > 80 else 100 if self.health > 50 else 50
        
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=max_tokens
            )
            self.successes += 1
            self.health = min(100, self.health + 5)
            return response.choices[0].message.content, None
        except Exception as e:
            self.errors += 1
            self.health = max(0, self.health - 20)
            return None, str(e)
    
    def status(self):
        state = "Healthy" if self.health > 80 else "Degraded" if self.health > 50 else "Critical"
        return f"{state} ({self.health}%), Errors: {self.errors}, Successes: {self.successes}"

# Demo
caller = ResilientCaller()
result, error = caller.call([{"role": "user", "content": "Hello"}])
print(f"Result: {result}")
print(f"Status: {caller.status()}")

---
## Exercise 8.4.3: Circuit Breaker

Implement the circuit breaker pattern for API calls.

In [ ]:
from enum import Enum

class State(Enum):
    CLOSED = "CLOSED"
    OPEN = "OPEN"
    HALF_OPEN = "HALF_OPEN"

class CircuitBreaker:
    def __init__(self, threshold=3, timeout=30):
        self.state = State.CLOSED
        self.failures = 0
        self.threshold = threshold
        self.timeout = timeout
        self.last_failure = None
    
    def call(self, messages):
        if self.state == State.OPEN:
            if self.last_failure and (datetime.now() - self.last_failure).seconds >= self.timeout:
                self.state = State.HALF_OPEN
            else:
                return None, "Circuit OPEN"
        
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            if self.state == State.HALF_OPEN:
                self.state = State.CLOSED
            self.failures = 0
            return response.choices[0].message.content, None
        except Exception as e:
            self.failures += 1
            self.last_failure = datetime.now()
            if self.failures >= self.threshold:
                self.state = State.OPEN
            return None, str(e)

# Demo
cb = CircuitBreaker()
result, error = cb.call([{"role": "user", "content": "Hello"}])
print(f"Result: {result}")
print(f"State: {cb.state.value}")

---
## Exercise 8.5.1: Topic Tracker

Create a class that tracks conversation topics.

In [ ]:
from collections import Counter

class TopicTracker:
    def __init__(self):
        self.topics = Counter()
        self.keywords = {
            'programming': ['code', 'python', 'function', 'variable'],
            'ai': ['ai', 'machine learning', 'neural', 'model'],
            'data': ['data', 'database', 'sql', 'analysis']
        }
    
    def analyze(self, message):
        msg_lower = message.lower()
        detected = []
        for topic, kws in self.keywords.items():
            if any(kw in msg_lower for kw in kws):
                detected.append(topic)
                self.topics[topic] += 1
        return detected or ['general']
    
    def report(self):
        if not self.topics:
            return "No topics tracked"
        total = sum(self.topics.values())
        return "\n".join(f"{t}: {c/total*100:.0f}%" for t, c in self.topics.most_common())

# Demo
tracker = TopicTracker()
tracker.analyze("How do I write Python code?")
tracker.analyze("What is machine learning?")
print(tracker.report())

---
## Exercise 8.5.2: Response Timer Class

Build a class that monitors API response performance.

In [ ]:
import time

class ResponseTimer:
    def __init__(self, slow_threshold=2.0):
        self.times = []
        self.slow_threshold = slow_threshold
    
    def start(self):
        return time.time()
    
    def end(self, start):
        elapsed = time.time() - start
        self.times.append(elapsed)
        return elapsed
    
    def stats(self):
        if not self.times:
            return "No data"
        return {
            'avg': sum(self.times) / len(self.times),
            'min': min(self.times),
            'max': max(self.times),
            'slow': sum(1 for t in self.times if t > self.slow_threshold)
        }

# Demo
timer = ResponseTimer()
start = timer.start()
client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hi"}])
print(f"Elapsed: {timer.end(start):.2f}s")
print(f"Stats: {timer.stats()}")

---
## Exercise 8.5.3: Mood Detector

Create a class that detects user mood and adjusts responses.

In [ ]:
class MoodDetector:
    def __init__(self):
        self.moods = {
            'happy': (['happy', 'great', 'awesome', 'love'], 'enthusiastic'),
            'sad': (['sad', 'depressed', 'unhappy'], 'supportive'),
            'angry': (['angry', 'frustrated', 'annoyed'], 'calm'),
            'confused': (['confused', 'unclear', 'lost'], 'patient')
        }
        self.current = 'neutral'
    
    def detect(self, message):
        msg_lower = message.lower()
        for mood, (words, _) in self.moods.items():
            if any(w in msg_lower for w in words):
                self.current = mood
                return mood
        self.current = 'neutral'
        return 'neutral'
    
    def style(self):
        if self.current in self.moods:
            return self.moods[self.current][1]
        return 'balanced'

# Demo
detector = MoodDetector()
for msg in ["I'm frustrated!", "This is awesome!", "Help me please"]:
    mood = detector.detect(msg)
    print(f"'{msg[:20]}...' -> {mood} (be {detector.style()})")

---
## Exercise 8.6.1: Export Master

Create a class that exports conversations to multiple formats.

In [ ]:
import csv

class ExportMaster:
    def to_html(self, messages, filename="export.html"):
        html = "<html><body><h1>Conversation</h1>"
        for m in messages:
            label = "You" if m['role'] == 'user' else "AI"
            html += f"<p><b>{label}:</b> {m['content']}</p>"
        html += "</body></html>"
        with open(filename, 'w') as f:
            f.write(html)
        return filename
    
    def to_csv(self, messages, filename="export.csv"):
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Role', 'Content', 'Words'])
            for m in messages:
                writer.writerow([m['role'], m['content'], len(m['content'].split())])
        return filename
    
    def to_markdown(self, messages, filename="export.md"):
        with open(filename, 'w') as f:
            f.write("# Conversation\n\n")
            for m in messages:
                label = "You" if m['role'] == 'user' else "AI"
                f.write(f"### {label}\n{m['content']}\n\n")
        return filename

# Demo
sample = [{"role": "user", "content": "Hi"}, {"role": "assistant", "content": "Hello!"}]
exporter = ExportMaster()
print(f"HTML: {exporter.to_html(sample)}")
print(f"CSV: {exporter.to_csv(sample)}")

---
## Exercise 8.6.2: Smart Organizer

Build a class that organizes conversations by topic and date.

In [ ]:
import json
from pathlib import Path
from datetime import datetime

class SmartOrganizer:
    def __init__(self, storage="organized"):
        self.storage = Path(storage)
        self.storage.mkdir(exist_ok=True)
    
    def detect_topic(self, messages):
        keywords = {'programming': ['code', 'python'], 'ai': ['ai', 'learning']}
        for m in messages:
            if m['role'] == 'user':
                for topic, kws in keywords.items():
                    if any(kw in m['content'].lower() for kw in kws):
                        return topic
        return 'general'
    
    def organize(self, messages):
        topic = self.detect_topic(messages)
        topic_dir = self.storage / topic
        topic_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = topic_dir / f"conv_{timestamp}.json"
        
        with open(filepath, 'w') as f:
            json.dump({'topic': topic, 'messages': messages}, f)
        
        return {'topic': topic, 'file': str(filepath)}

# Demo
organizer = SmartOrganizer()
sample = [{"role": "user", "content": "How do I write Python code?"}]
print(organizer.organize(sample))

---
## Exercise 8.6.3: History Analytics

Create a class that analyzes conversation history for insights.

In [ ]:
from collections import Counter
import statistics

class HistoryAnalytics:
    def __init__(self):
        self.conversations = []
    
    def add(self, messages):
        self.conversations.append({'messages': messages, 'count': len(messages)})
    
    def topics(self):
        stop_words = {'the', 'is', 'a', 'to', 'for', 'what', 'how'}
        words = Counter()
        for conv in self.conversations:
            for m in conv['messages']:
                if m['role'] == 'user':
                    for w in m['content'].lower().split():
                        if len(w) > 3 and w not in stop_words:
                            words[w.strip('.,!?')] += 1
        return words.most_common(5)
    
    def lengths(self):
        counts = [c['count'] for c in self.conversations]
        if not counts:
            return "No data"
        return {'avg': statistics.mean(counts), 'min': min(counts), 'max': max(counts)}

# Demo
analytics = HistoryAnalytics()
analytics.add([{"role": "user", "content": "How do I learn Python?"}])
analytics.add([{"role": "user", "content": "Explain Python functions"}])
print(f"Topics: {analytics.topics()}")
print(f"Lengths: {analytics.lengths()}")

---
## Summary

In this chapter you learned to build LLM applications with:
- Basic API calls and chatbots
- Temperature and token management
- Error handling and resilience patterns
- Conversation analysis and export

Next: **Chapter 9: Prompt Engineering**